In [175]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from data_handler import execute_sql_script_from_config
from lookups import FileType
from prehook import return_csv_list,create_staging_tables
import prehook

db_session = create_connection()

In [176]:
execute_sql_script_from_config('SQL_Commands/Create_Staging_Tables.sql')
execute_sql_script_from_config('SQL_Commands/V10_hook_create_agg_playlist.sql')


Error executing SQL script: relation "stg_kaggle_spotify_tracks" already exists

SQL script executed successfully.


In [177]:
def insert_statements_from_dataframe(dataframe, schema_name, table_name):
    insert_statements = []

    for _, row in dataframe.iterrows():
        formatted_row = []

        for value in row:
            if pd.notna(value):
                if isinstance(value, str):
                    formatted_value = value.replace("'", "''")
                    formatted_row.append(f"'{formatted_value}'")
                elif isinstance(value, pd.Timestamp):
                    formatted_row.append(f"'{value}'")
                else:
                    formatted_row.append(str(value))
            else:
                formatted_row.append('NULL')

        values = ', '.join(formatted_row)
        columns = ', '.join(dataframe.columns)
        insert_statement = f"INSERT INTO {schema_name}.{table_name} ({columns}) VALUES ({values});"
        insert_statements.append(insert_statement)

    return insert_statements

df2 = pd.read_excel("C:\\dataproject\\users_dataset.xlsx")
insert_statements = insert_statements_from_dataframe(df2, 'musicschema', 'stg_kaggle_spotify_users')

with db_session.cursor() as cursor:
    for statement in insert_statements:
        cursor.execute(statement)

# Commit the changes
db_session.commit()



UniqueViolation: duplicate key value violates unique constraint "stg_kaggle_spotify_users_pkey"
DETAIL:  Key (id)=(20000) already exists.


In [ ]:
def insert_statements_from_dataframe(dataframe, schema_name, table_name):
    insert_statements = []

    for _, row in dataframe.iterrows():
        formatted_row = []

        for value in row:
            if pd.notna(value):
                if isinstance(value, str):
                    formatted_value = value.replace("'", "''")
                    formatted_row.append(f"'{formatted_value}'")
                elif isinstance(value, pd.Timestamp):
                    formatted_row.append(f"'{value}'")
                else:
                    formatted_row.append(str(value))
            else:
                formatted_row.append('NULL')

        values = ', '.join(formatted_row)
        columns = ', '.join(dataframe.columns)
        insert_statement = f"INSERT INTO {schema_name}.{table_name} ({columns}) VALUES ({values});"
        insert_statements.append(insert_statement)

    return insert_statements

df2 = pd.read_excel("C:\\dataproject\\songs_dataset.xlsx")
insert_statements = insert_statements_from_dataframe(df2, 'musicschema', 'stg_kaggle_spotify_tracks')

with db_session.cursor() as cursor:
    for statement in insert_statements:
        cursor.execute(statement)

# Commit the changes
db_session.commit()



UniqueViolation: duplicate key value violates unique constraint "stg_kaggle_spotify_tracks_pkey"
DETAIL:  Key (track_id)=(3121) already exists.


In [ ]:
import psycopg2
import json

file = 'SQL_Commands/V2_prehook_Create_ETL_watermark_table.sql'
#execute_sql_statements(db_session,file)

# Load database connection details from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname=config_data['database'],
    user=config_data['user'],
    password=config_data['password'],
    host=config_data['host'],
    port=config_data['port']
)

# Create a cursor object
cursor = conn.cursor()

# Read the SQL file
with open('SQL_Commands/V2_prehook_Create_ETL_watermark_table.sql', 'r') as sql_file:
    sql_script = sql_file.read()

# Execute the SQL script
cursor.execute(sql_script)

# Commit the changes
conn.commit()

# Close the connection
conn.close()
